In [ ]:
!pip install split_folders

imports

In [ ]:
import tensorflow as tf
import sklearn
import splitfolders
import os
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

goes through copied dataset and removes gdot files

In [ ]:
input_folder = "/content/drive/MyDrive/machine vision/copieddata"

In [ ]:
#does not need to be ran again
for class_dir in os.listdir(input_folder):
    class_path = os.path.join(input_folder, class_dir)
    if not os.path.isdir(class_path):
        continue

    for filename in os.listdir(class_path):
        if filename.startswith('.') or not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(class_path, filename)
            print(f"Removing: {file_path}")
            os.remove(file_path)

copies dataset into new folder splitting the data into 80% training 20% test

In [ ]:
#does not need to be executed again
output = "/content/drive/MyDrive/machine vision/splitdata" #Enter Output Folder

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.8,0.2))

Copying files: 8 files [00:22,  1.93s/ files]

KeyboardInterrupt: 

source: https://stackoverflow.com/questions/57394135/split-image-dataset-into-train-test-datasets

creates paths so that parts of dataset can be refrenced

In [ ]:
#does not need to be executed again
#create a zip of the dataset in Google Drive
!zip -r "/content/drive/MyDrive/machine_vision_dataset.zip" "/content/drive/MyDrive/machine vision/splitdata"


Scanning files .....


zip error: Interrupted (aborting)


copies zipped to local colab cache so that it is faster to refrence

In [ ]:

!cp "/content/drive/MyDrive/machine_vision_dataset.zip" /content/
!unzip /content/machine_vision_dataset.zip -d /content/


Streaming output truncated to the last 5000 lines.
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_601.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_766.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_719.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_73.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_773.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_688.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_712.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_587.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_68.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/splitdata/val/JN/JN_742.jpg  
  inflating: /content/content/drive/MyDrive/machine vision/sp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_path   = '/content/content/drive/MyDrive/machine vision/splitdata'
train_path = '/content/content/drive/MyDrive/machine vision/splitdata/train'  #creates paths for training and testing
validity_path = '/content/content/drive/MyDrive/machine vision/splitdata/val'

creates both datasets so that all images are resized in them and uses batches as trying to train on all the images all at once breaks model, creates class names form folders

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=(224, 224),  # resizes pictures
    batch_size=32
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    validity_path,
    image_size=(224, 224),
    batch_size=32
)

class_names = train_dataset.class_names
print(class_names)  #creates class names from folders



Found 21935 files belonging to 75 classes.
Found 5488 files belonging to 75 classes.
['B', 'CC', 'CL', 'CM', 'CN', 'CS', 'CU', 'CXD', 'CXI', 'D', 'DB', 'DEC', 'DEE', 'DEF', 'DEG_1', 'DEG_2', 'DER', 'DES', 'EX', 'FC', 'FL', 'FM', 'FS', 'H', 'ID', 'IG', 'ING', 'INGF', 'INGZ', 'IR', 'IS', 'JDL', 'JDM_1', 'JDM_2', 'JN', 'JX', 'JXB', 'JXD', 'LD', 'LL', 'LR', 'LU', 'LX', 'MB', 'MC', 'MM', 'OB', 'OBB', 'OBC', 'OBI', 'OBM', 'OBP', 'OBS', 'OBX', 'OJL', 'OJM', 'RF', 'RM_1', 'RM_2', 'RT', 'SAP', 'SAV', 'SCP', 'SR', 'SRC', 'SRP', 'SRV', 'SS', 'SV', 'SW', 'VR', 'VV', 'WL', 'XB', 'XP']


creates class weights as each class does not have the same amount of pictures

In [ ]:
def calculate_class_weights(train_dataset):
    label_list = []

    for _, label_batch in train_dataset:
        label_list.extend(label_batch.numpy())  # append batch labels

    label_counts = Counter(label_list)
    total_samples = sum(label_counts.values())
    num_classes = len(label_counts)

    class_weights = {
        class_id: total_samples / (num_classes * count)
        for class_id, count in label_counts.items()
    }

    return class_weights

# Compute class weights
class_weights = calculate_class_weights(train_dataset)

# Print sample weights
for k in list(class_weights.keys())[:5]:
    print(f"Class {k}: weight {class_weights[k]:.4f}")

Class 40: weight 0.3733
Class 44: weight 0.9461
Class 8: weight 0.4867
Class 12: weight 0.3770
Class 13: weight 0.4383


do not run

In [ ]:
#train_dataset = train_dataset.shuffle(1000).prefetch(tf.data.AUTOTUNE)
#val_dataset = val_dataset.cache().prefetch(tf.data.AUTOTUNE)

used to run faster

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')  #uses both 16 and 32 bit to run faster and use less memory


creates CNN

In [ ]:
num_classes = 75

base_model = tf.keras.applications.EfficientNetB4(  #creates model using efficientnet
    include_top=False,
    input_shape=(224, 224, 3),  #decreased image size
    pooling='avg',
    weights='imagenet'
)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomZoom(0.15),
    tf.keras.layers.RandomContrast(0.15),
    tf.keras.layers.RandomTranslation(0.1, 0.1) #augmentations to make images more random which should result in better accuracy
])                                              #as CNN trains on more images

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    data_augmentation,
    base_model,                                     #creates model
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, dtype='float32'),
])

In [ ]:
for images, labels in train_dataset.take(1):
    print("Train batch shape:", images.shape)
    print("Train labels:", labels.numpy())


Train batch shape: (32, 224, 224, 3)
Train labels: [17  1 20 12 57 32  1 13  9  8 20 11  0 38 38 56 39 20 39 16 39 56  7  8
 23 20 16 45  1 12 72 23]


compiles model

In [ ]:
model.compile(
       optimizer='adam', #compiles model
       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
       metrics=['accuracy'])

history = model.fit(train_dataset, validation_data=val_dataset, epochs=15)

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


model compiles however accuracy is at 81%, must use GPU instead of CPU when compiling, training also takes a long time
